In [1]:
# set working directory 
get_ipython().magic(u'pwd')
get_ipython().magic(u'cd "C:/Users/Wayne/Dropbox/Acer Laptop Sync/Data Science/GFW Visualization"')

import sys
import json
import sqlite3
import time
import datetime
from pprint import pprint
import string
import itertools

import unicodedata   

from numpy import random, asarray, linspace, corrcoef
import pandas as pd
from pandas import Series

import jupyter_client
import ipykernel

from lightning import Lightning

import sqlalchemy
from sqlalchemy.orm import mapper, sessionmaker
from sqlalchemy import Column, Integer, String, ForeignKey, Text, DateTime, Unicode
from sqlalchemy.ext.declarative import declarative_base
from types import *
from sqlalchemy import func     #TO PERFORM 'COUNT' QUERIES
from sqlalchemy import distinct #TO COUNT 'DISTINCT' VALUES IN DB
from sqlalchemy import create_engine

%matplotlib inline 
import matplotlib.pyplot as plt
from pylab import* 

from table_classes_users_followers_v2 import * # make sure this .py file exists in your working directory

# connect to an existing database
engine = sqlalchemy.create_engine("sqlite:///C:/Users/Wayne/Dropbox/Acer Laptop Sync/Data Science/GFW Visualization/GFW_tweets starting 9-1-2015.sqlite", echo=False)   #ABSOLUTE PATH TO DB       
Session = sessionmaker(bind=engine)
session = Session()  
Base.metadata.create_all(engine)
all_tweets = session.query(TWEET).all()

con = sqlite3.connect("C:/Users/Wayne/Dropbox/Acer Laptop Sync/Data Science/GFW Visualization/GFW_tweets starting 9-1-2015.sqlite")
cur = con.cursor()
df = pd.read_sql_query("SELECT * from gfw_tweets", con)
con.close()

df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].apply(lambda x: x.date())
df = df.loc[df['query'].isin(['%23FuckGFW','%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91','%E7%BF%BB%E5%A2%99','%E9%98%B2%E7%81%AB%E5%A2%99','greatfirewall'])]

print ("Total # of tweets in SQLite:", len(all_tweets))
print ("Total # of tweets in filtered PANDAS dataframe:", len(df))

df = df.sort(['created_at'])
df = df.set_index(['created_at'])

def fuckgfwcumsum(row):
    if row['query'] == "%23FuckGFW":
        return 1
    return 0

def kxswcumsum(row):
    if row['query'] == "%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91":
        return 1
    return 0

def fqcumsum(row):
    if row['query'] == "%E7%BF%BB%E5%A2%99":
        return 1
    return 0

def fhqcumsum(row):
    if row['query'] == "%E9%98%B2%E7%81%AB%E5%A2%99":
        return 1
    return 0

def greatfirewallcumsum(row):
    if row['query'] == "greatfirewall":
        return 1
    return 0

df['fuckgfw_base_tweet_count'] = df.apply(lambda row: fuckgfwcumsum(row), axis=1)    
df['fuckgfw_cum_tweet_count'] = df.fuckgfw_base_tweet_count.cumsum()
df['kxsw_base_tweet_count'] = df.apply(lambda row: kxswcumsum(row), axis=1)    
df['kxsw_cum_tweet_count'] = df.kxsw_base_tweet_count.cumsum()
df['fq_base_tweet_count'] = df.apply(lambda row: fqcumsum(row), axis=1)    
df['fq_cum_tweet_count'] = df.fq_base_tweet_count.cumsum()
df['fhq_base_tweet_count'] = df.apply(lambda row: fhqcumsum(row), axis=1)    
df['fhq_cum_tweet_count'] = df.fhq_base_tweet_count.cumsum()
df['greatfirewall_base_tweet_count'] = df.apply(lambda row: greatfirewallcumsum(row), axis=1)    
df['greatfirewall_cum_tweet_count'] = df.greatfirewall_base_tweet_count.cumsum()

def dailycount(x):
            return Series(dict(Number_of_tweets = x['content'].count(),
                        ))

fuckgfw_daily_count = df[df['query'] == '%23FuckGFW'].groupby(df[df['query'] == '%23FuckGFW'].index).apply(dailycount)
kxsw_daily_count = df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].groupby(df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].index).apply(dailycount)
fq_daily_count = df[df['query'] == '%E7%BF%BB%E5%A2%99'].groupby(df[df['query'] == '%E7%BF%BB%E5%A2%99'].index).apply(dailycount)
fhq_daily_count = df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].groupby(df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].index).apply(dailycount)
greatfirewall_daily_count = df[df['query'] == 'greatfirewall'].groupby(df[df['query'] == 'greatfirewall'].index).apply(dailycount)

fuckgfw_daily_count['fuckgfw_count'] =  fuckgfw_daily_count['Number_of_tweets']
kxsw_daily_count['kxsw_count'] = kxsw_daily_count['Number_of_tweets']
fq_daily_count['fq_count'] = fq_daily_count['Number_of_tweets']
fhq_daily_count['fhq_count'] = fhq_daily_count['Number_of_tweets']
greatfirewall_daily_count['greatfirewall_count'] = greatfirewall_daily_count['Number_of_tweets']

dailycount_forall = pd.concat([fuckgfw_daily_count, kxsw_daily_count, fq_daily_count, fhq_daily_count,greatfirewall_daily_count], axis=1)
dailycount_forall = dailycount_forall[['fuckgfw_count', 'kxsw_count','fq_count','fhq_count','greatfirewall_count']]
dailycount_forall = dailycount_forall.fillna(0)

def dailyrtcount(x):
    return Series(dict(Number_of_rts = x['retweet_count'].sum(),
                        ))

def fuckgfwvirality(x):
    return x['fuckgfw_rtcount']/x['fuckgfw_count']

def kxswvirality(x):
    return x['kxsw_rtcount']/x['kxsw_count']

def fqvirality(x):
    return x['fq_rtcount']/x['fq_count']

def fhqvirality(x):
    return x['fhq_rtcount']/x['fhq_count']

def greatfirewallvirality(x):
    return x['greatfirewall_rtcount']/x['greatfirewall_count']

fuckgfw_rt_count = df[df['query'] == '%23FuckGFW'].groupby(df[df['query'] == '%23FuckGFW'].index).apply(dailyrtcount)
kxsw_rt_count = df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].groupby(df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].index).apply(dailyrtcount)
fq_rt_count = df[df['query'] == '%E7%BF%BB%E5%A2%99'].groupby(df[df['query'] == '%E7%BF%BB%E5%A2%99'].index).apply(dailyrtcount)
fhq_rt_count = df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].groupby(df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].index).apply(dailyrtcount)
greatfirewall_rt_count = df[df['query'] == 'greatfirewall'].groupby(df[df['query'] == 'greatfirewall'].index).apply(dailyrtcount)

fuckgfw_rt_count['fuckgfw_rtcount'] =  fuckgfw_rt_count['Number_of_rts']
kxsw_rt_count['kxsw_rtcount'] = kxsw_rt_count['Number_of_rts']
fq_rt_count['fq_rtcount'] = fq_rt_count['Number_of_rts']
fhq_rt_count['fhq_rtcount'] = fhq_rt_count['Number_of_rts']
greatfirewall_rt_count['greatfirewall_rtcount'] = greatfirewall_rt_count['Number_of_rts']

#merge1 = pd.merge(fuckgfw_daily_count, kxsw_daily_count, how='inner', on= None, left_on=None, right_on=None,
#      left_index=None, right_index=False, sort=True, suffixes=('_x', '_y'), copy=True)
#print len(fuckgfw_daily_count)
#merge1 
rtcount_forall = pd.concat([fuckgfw_rt_count, kxsw_rt_count, fq_rt_count, fhq_rt_count,greatfirewall_rt_count ], axis=1)
rtcount_forall = rtcount_forall[['fuckgfw_rtcount', 'kxsw_rtcount','fq_rtcount','fhq_rtcount','greatfirewall_rtcount']]
rtcount_forall = rtcount_forall.fillna(0)
rtandcountcombined = pd.concat([rtcount_forall,dailycount_forall], axis=1)
rtandcountcombined['fuckgfw_virality'] = rtandcountcombined.apply(lambda x:fuckgfwvirality(x), axis=1)
rtandcountcombined['kxsq_virality'] = rtandcountcombined.apply(lambda x:kxswvirality(x), axis=1)
rtandcountcombined['fq_virality'] = rtandcountcombined.apply(lambda x:fqvirality(x), axis=1)
rtandcountcombined['fhq_virality'] = rtandcountcombined.apply(lambda x:fhqvirality(x), axis=1)
rtandcountcombined['greatfirewall_virality'] = rtandcountcombined.apply(lambda x:greatfirewallvirality(x), axis=1)
virality_display = rtandcountcombined[['fuckgfw_virality','kxsq_virality','fq_virality','fhq_virality','greatfirewall_virality']]

def dailyfollowercount(x):
    return Series(dict(Number_of_followers = x['from_user_followers_count'].sum(),
                        ))

fuckgfw_follower_count = df[df['query'] == '%23FuckGFW'].groupby(df[df['query'] == '%23FuckGFW'].index).apply(dailyfollowercount)
kxsw_follower_count = df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].groupby(df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].index).apply(dailyfollowercount)
fq_follower_count = df[df['query'] == '%E7%BF%BB%E5%A2%99'].groupby(df[df['query'] == '%E7%BF%BB%E5%A2%99'].index).apply(dailyfollowercount)
fhq_follower_count = df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].groupby(df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].index).apply(dailyfollowercount)
greatfirewall_follower_count = df[df['query'] == 'greatfirewall'].groupby(df[df['query'] == 'greatfirewall'].index).apply(dailyfollowercount)

fuckgfw_follower_count['fuckgfw_followercount'] =  fuckgfw_follower_count['Number_of_followers']
kxsw_follower_count['kxsw_followercount'] = kxsw_follower_count['Number_of_followers']
fq_follower_count['fq_followercount'] = fq_follower_count['Number_of_followers']
fhq_follower_count['fhq_followercount'] = fhq_follower_count['Number_of_followers']
greatfirewall_follower_count['greatfirewall_followercount'] = greatfirewall_follower_count['Number_of_followers']

followercount_forall = pd.concat([fuckgfw_follower_count, kxsw_follower_count, fq_follower_count, fhq_follower_count,greatfirewall_follower_count ], axis=1)
followercount_forall = followercount_forall[['fuckgfw_followercount', 'kxsw_followercount','fq_followercount','fhq_followercount','greatfirewall_followercount']]
followercount_forall = followercount_forall.fillna(0)

def dailydiscussantscount(x):
    return Series(dict(Number_of_discussants = len(pd.unique(x['from_user_screen_name'].ravel())),
                        ))

fuckgfw_discussant_count = df[df['query'] == '%23FuckGFW'].groupby(df[df['query'] == '%23FuckGFW'].index).apply(dailydiscussantscount)
kxsw_discussant_count = df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].groupby(df[df['query'] == '%E7%A7%91%E5%AD%A6%E4%B8%8A%E7%BD%91'].index).apply(dailydiscussantscount)
fq_discussant_count = df[df['query'] == '%E7%BF%BB%E5%A2%99'].groupby(df[df['query'] == '%E7%BF%BB%E5%A2%99'].index).apply(dailydiscussantscount)
fhq_discussant_count = df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].groupby(df[df['query'] == '%E9%98%B2%E7%81%AB%E5%A2%99'].index).apply(dailydiscussantscount)
greatfirewall_discussant_count = df[df['query'] == 'greatfirewall'].groupby(df[df['query'] == 'greatfirewall'].index).apply(dailydiscussantscount)

fuckgfw_discussant_count['fuckgfw_discussantcount'] =  fuckgfw_discussant_count['Number_of_discussants']
kxsw_discussant_count['kxsw_discussantcount'] = kxsw_discussant_count['Number_of_discussants']
fq_discussant_count['fq_discussantcount'] = fq_discussant_count['Number_of_discussants']
fhq_discussant_count['fhq_discussantcount'] = fhq_discussant_count['Number_of_discussants']
greatfirewall_discussant_count['greatfirewall_discussantcount'] = greatfirewall_discussant_count['Number_of_discussants']

discussant_forall = pd.concat([fuckgfw_discussant_count, kxsw_discussant_count, fq_discussant_count, fhq_discussant_count,greatfirewall_discussant_count ], axis=1)
discussant_forall = discussant_forall[['fuckgfw_discussantcount', 'kxsw_discussantcount','fq_discussantcount','fhq_discussantcount','greatfirewall_discussantcount']]
discussant_forall = discussant_forall.fillna(0)

def fuckgfwaudiencesize(x):
    return x['fuckgfw_followercount']/x['fuckgfw_discussantcount']

def kxswaudiencesize(x):
    return x['kxsw_followercount']/x['kxsw_discussantcount']

def fqaudiencesize(x):
    return x['fq_followercount']/x['fq_discussantcount']

def fhqaudiencesize(x):
    return x['fhq_followercount']/x['fhq_discussantcount']

def greatfirewallaudiencesize(x):
    return x['greatfirewall_followercount']/x['greatfirewall_discussantcount']

audiencesizecombined = pd.concat([discussant_forall,followercount_forall], axis=1)
audiencesizecombined['fuckgfw_audiencesize'] = audiencesizecombined.apply(lambda x: fuckgfwaudiencesize(x), axis=1)
audiencesizecombined['kxsw_audiencesize'] = audiencesizecombined.apply(lambda x:kxswaudiencesize(x), axis=1)
audiencesizecombined['fq_audiencesize'] = audiencesizecombined.apply(lambda x:fqaudiencesize(x), axis=1)
audiencesizecombined['fhq_audiencesize'] = audiencesizecombined.apply(lambda x:fhqaudiencesize(x), axis=1)
audiencesizecombined['greatfirewall_audiencesize'] = audiencesizecombined.apply(lambda x:greatfirewallaudiencesize(x), axis=1)
audiencesize_display = audiencesizecombined[['fuckgfw_audiencesize','kxsw_audiencesize','fq_audiencesize','fhq_audiencesize','greatfirewall_audiencesize']]
audiencesize_display = audiencesize_display.fillna(0)
audiencesize_display['fuckgfw_audiencesize'] = audiencesize_display['fuckgfw_audiencesize'].astype(int)
audiencesize_display['kxsw_audiencesize'] = audiencesize_display['kxsw_audiencesize'].astype(int)
audiencesize_display['fq_audiencesize'] = audiencesize_display['fq_audiencesize'].astype(int)
audiencesize_display['fhq_audiencesize'] = audiencesize_display['fhq_audiencesize'].astype(int)
audiencesize_display['greatfirewall_audiencesize'] = audiencesize_display['greatfirewall_audiencesize'].astype(int)

C:\Users\Wayne\Dropbox\Acer Laptop Sync\Data Science\GFW Visualization
('Total # of tweets in SQLite:', 40283)
('Total # of tweets in filtered PANDAS dataframe:', 22326)


,#fuckgfw,Ke Xue Shang Wang (科学上网),Fan Qiang (翻墙),Fang Huo Qiang （防火墙）,#greatfirewall
2015-08-24,0,1,35,0,0
2015-08-25,5,2,437,26,2
2015-08-26,3,6,358,19,11
2015-08-27,11,6,445,54,3
2015-08-28,8,5,419,49,9
2015-08-29,5,16,322,19,0
2015-08-30,2,6,447,18,7
2015-08-31,2,2,551,71,6
2015-09-01,11,22,342,138,4
2015-09-02,6,4,605,101,6


In [2]:
#dailycount_forall
dailycount_forall.columns = ['#fuckgfw', 'Ke Xue Shang Wang (科学上网)', "Fan Qiang (翻墙)", "Fang Huo Qiang （防火墙）", "#greatfirewall"]
dailycount_forall

# create Twitter Activity Chart
import plotly.plotly as py
from plotly.graph_objs import *
import cufflinks as cf
py.sign_in('cosmopolitanvan', 'jnv1ti79zi')

layout = Layout(
    title="Twitter Activity of Keywords/Hashtags related to China's Great Firewall",
    titlefont=Font(
        family='Arial, sans-serif',
        size=22,
        color='black'
    ),
    font=Font(
        family='"Droid Sans", sans-serif',
        size=12
    ),
    showlegend=True,
    autosize=False,
    width=850,
    height=430,
    xaxis=XAxis(
        title='',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[1440388800000, 1442462400000],
        type='date',
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='inside',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    yaxis=YAxis(
        title='# of tweets',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[-32.55555555555556, 638.5555555555555],
        type='linear',
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    legend=Legend(
        x=0.9013254786450663,
        y=1.132,
        traceorder='normal',
        font=Font(
            family='Arial, sans-serif',
            size=12,
            color='#666666'
        ),
        bgcolor='rgb(255, 255, 255)',
        bordercolor='black',
        borderwidth=0,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='top'
    ),
    paper_bgcolor='rgb(213, 226, 233)',
    plot_bgcolor='rgb(213, 226, 233)',
    bargap=0.2
)

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

dailycount_forall.iplot(kind='scatter', layout = layout, filename='GFW/activity1')

C:\Users\Wayne\Anaconda\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [10]:
virality_display

,#fuckgfw,Ke Xue Shang Wang (科学上网),Fan Qiang (翻墙),Fang Huo Qiang （防火墙）,#greatfirewall
2015-08-24,NaN,0.000000,37.628571,NaN,NaN
2015-08-25,0.200000,0.000000,35.894737,54.961538,0.000000
2015-08-26,16.000000,0.666667,11.843575,87.368421,1.363636
2015-08-27,0.727273,3.000000,12.523596,58.425926,0.000000
2015-08-28,1.500000,0.000000,9.668258,41.081633,0.888889
2015-08-29,0.600000,2.500000,7.347826,5.000000,NaN
2015-08-30,0.000000,0.166667,24.375839,143.166667,0.428571
2015-08-31,0.000000,0.000000,18.531760,70.507042,0.000000
2015-09-01,9.272727,19.090909,13.140351,73.101449,0.500000
2015-09-02,0.000000,5.500000,37.479339,66.128713,3.333333


In [3]:
#dailycount_forall
virality_display.columns = ['#fuckgfw', 'Ke Xue Shang Wang (科学上网)', "Fan Qiang (翻墙)", "Fang Huo Qiang （防火墙）", "#greatfirewall"]

import plotly.plotly as py
from plotly.graph_objs import *
import cufflinks as cf
py.sign_in('cosmopolitanvan', 'jnv1ti79zi')

layout = Layout(
    title="Twitter Virality of Keywords/Hashtags related to China's Great Firewall",
    titlefont=Font(
        family='Arial, sans-serif',
        size=22,
        color='black'
    ),
    font=Font(
        family='"Droid Sans", sans-serif',
        size=12
    ),
    showlegend=True,
    autosize=False,
    width=850,
    height=430,
    xaxis=XAxis(
        title='',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[1440388800000, 1442462400000],
        type='date',
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='inside',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    yaxis=YAxis(
        title='Retweet count per tweet',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[-32.55555555555556, 638.5555555555555],
        type='linear',
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=10,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    legend=Legend(
        x=0.9013254786450663,
        y=1.132,
        traceorder='normal',
        font=Font(
            family='Arial, sans-serif',
            size=12,
            color='#666666'
        ),
        bgcolor='rgb(255, 255, 255)',
        bordercolor='black',
        borderwidth=0,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='top'
    ),
    paper_bgcolor='rgb(213, 226, 233)',
    plot_bgcolor='rgb(213, 226, 233)',
    bargap=0.2
)

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

virality_display.iplot(kind='scatter', layout = layout, filename='GFW/virality')

C:\Users\Wayne\Anaconda\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [4]:
audiencesize_display.columns = ['#fuckgfw', 'Ke Xue Shang Wang (科学上网)', "Fan Qiang (翻墙)", "Fang Huo Qiang （防火墙）", "#greatfirewall"]
import plotly.plotly as py
from plotly.graph_objs import *
import cufflinks as cf
py.sign_in('cosmopolitanvan', 'jnv1ti79zi')

layout = Layout(
    title="Twitter Audience Size of Keywords/Hashtags related to China's Great Firewall",
    titlefont=Font(
        family='Arial, sans-serif',
        size=22,
        color='black'
    ),
    font=Font(
        family='"Droid Sans", sans-serif',
        size=12
    ),
    showlegend=True,
    autosize=False,
    width=850,
    height=430,
    xaxis=XAxis(
        title='',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[1440388800000, 1442462400000],
        type='date',
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='inside',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    yaxis=YAxis(
        title='Avg. follower count of discussant',
        titlefont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        range=[-32.55555555555556, 638.5555555555555],
        type='linear',
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        nticks=5,
        ticks='',
        ticklen=6,
        tickcolor='rgba(0, 0, 0, 0)',
        tickfont=Font(
            family='"Verdana", monospace',
            size=12,
            color='black'
        ),
        mirror=False,
        gridcolor='white',
        gridwidth=1,
        zerolinecolor='#F6F6F6',
        zerolinewidth=1,
        linecolor='rgba(152, 0, 0, 0.5)',
        linewidth=1.5
    ),
    legend=Legend(
        x=1.0613254786450663,
        y=1.132,
        traceorder='normal',
        font=Font(
            family='Arial, sans-serif',
            size=12,
            color='#666666'
        ),
        bgcolor='rgb(255, 255, 255)',
        bordercolor='black',
        borderwidth=0,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='top'
    ),
    paper_bgcolor='rgb(213, 226, 233)',
    plot_bgcolor='rgb(213, 226, 233)',
    bargap=0.2
)

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

audiencesize_display.iplot(kind='scatter', layout = layout, filename='GFW/audiencesize')


C:\Users\Wayne\Anaconda\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

